In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#pip install spotipy
import spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Configuración
CLIENT_ID = '###'
CLIENT_SECRET = '##'
REDIRECT_URI = 'http://localhost:8889'
SCOPE = 'playlist-modify-public'

# Autenticación con la API de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))

# Obtener el ID del usuario
user_id = sp.current_user()['id']

# Crear una nueva playlist
playlist_name = "Mi Playlist para meditar"
description = "Playlist creada desde Python con Spotipy"
playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=True, description=description)
print(f"Playlist '{playlist_name}' creada con éxito")



Playlist 'Mi Playlist para meditar' creada con éxito


In [25]:
#### Clustering model

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
VARS = [
    "duration",
    "popularity",
    "danceability",
    "energy",
    "loudness",
    "instrumentalness",
]
dataset = pd.read_excel("../data/dataset.xlsx")
train  = dataset[VARS]
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
modelo_kmeans = KMeans(n_clusters=8, random_state=123)
modelo_kmeans.fit(train_scaled)
dataset["cluster_kmeans"] = modelo_kmeans.labels_
df_grouped = dataset.groupby("cluster_kmeans").agg(
    {
        "duration": "mean",
        "popularity": "mean",
        "danceability": "mean",
        "energy": "mean",
        "loudness": "mean",
        "instrumentalness": "mean",
    }
).reset_index()

cluster_number = df_grouped.sort_values("loudness", ascending = True).head(1)["cluster_kmeans"].values[0]
track_id = list(dataset[dataset["cluster_kmeans"] == cluster_number].sort_values("loudness", ascending = True).head(20)['id'])


In [37]:
df_grouped

,cluster_kmeans,duration,popularity,danceability,energy,loudness,instrumentalness
0,0,388879.590164,46.606557,0.419131,0.567820,-9.329344,0.242351
1,1,213636.496970,1.587879,0.410192,0.208788,-13.913527,0.016946
2,2,251480.991870,15.219512,0.544512,0.212396,-17.237984,0.826496
3,3,224424.019900,60.616915,0.481527,0.628308,-7.408995,0.032457
4,4,196633.390756,72.634454,0.711424,0.740336,-5.843941,0.043936
5,5,207538.780822,9.157534,0.671356,0.372399,-10.844192,0.030632
6,6,642297.100000,37.550000,0.368350,0.443295,-12.624450,0.258124
7,7,266770.562500,47.900000,0.433087,0.180125,-16.398475,0.074109


In [30]:
import pickle
with open('../models/modelo_kmeans.pkl', 'wb') as file:
    pickle.dump(modelo_kmeans, file)

with open('../models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [26]:
# Añadir canciones a la playlist (usando IDs de canciones)

sp.playlist_add_items(playlist_id=playlist['id'], items=track_id)



{'snapshot_id': 'AAAAAqXvFJBuucRlCYhIyHe5sppc21ac'}